# Extracting and Uploading MaStR - Winddata with open-mastr

In [171]:
from open_mastr import Mastr
import pandas as pd
import geopandas as gpd
import supabase_py
import psycopg2
import sqlalchemy
import os
import numpy as np
from dotenv import load_dotenv
from pandas.api.types import is_datetime64_any_dtype
from datetime import datetime
import time

In [ ]:
tables_list = ["wind", "location", "permit"]
db = Mastr()
#db.download(method = "bulk",
#            data=tables_list)

In [ ]:
### Inspect db
conn = db.engine # Connection engine
tables = pd.read_sql_query('SELECT name from sqlite_master where type= "table";', conn)
df_wind = pd.read_sql_table("wind_extended", conn)
columns_wind = list(df_wind.columns)
del conn

In [ ]:
### Very few plants actually have an address -> Not useful for displaying?
df_address = df_wind[["Strasse", "StrasseNichtGefunden",
       "Hausnummer", "HausnummerNichtGefunden", "Adresszusatz"]]
df_address.info()

In [ ]:
columns_remove = ["Lage",	"Seelage",	"ClusterOstsee", "ClusterNordsee", "Strasse", "StrasseNichtGefunden",
       "Hausnummer", "HausnummerNichtGefunden", "Adresszusatz", "NetzbetreiberpruefungStatus", 
       "NetzbetreiberpruefungDatum", "Wassertiefe", "Kuestenentfernung", "UtmZonenwert","UtmEast", "UtmNorth",
 "GaussKruegerHoch", "GaussKruegerRechts", "DatenQuelle", "DatumDownload"]
columns_wind = [col for col in columns_wind if col not in columns_remove]
df_wind = df_wind[columns_wind]
columns_wind = df_wind.columns

In [ ]:
### function to turn CamelCase to snake_case
def change_case(str):
    # List comprehension, starts with an _ wich is removed by lstrip("_")
    # loops through word, if upper, _ first "_"+
    # and i.lower() as a string method
    # just return i else
    return ''.join(['_'+i.lower() if i.isupper() 
               else i for i in str]).lstrip('_')
    
def dtype_sqltype(str, map_dict):
    
    # next() jumps through the iterator until a match is found
    # with a an iterator generated by the comprehension inside ()
    return next((key for key, val in map_dict.items() if val == str), None)


In [ ]:
### Set schema_name and table_name
schema_name = "public"
table_name = "wind_extended"

In [ ]:
### SQL Create Statement
pd_types = [df_wind[col].dtype for col in columns_wind]

# dictionary mapping the data types: Postgres Data type = pandas data type
map_types = {'bool': 'bool', 
                 'float8': 'float64', 
                 'date' : '<M8[ns]', 
                 'varchar':'O'}

# column-names for coordinates
geo_columns = ['Laengengrad','Breitengrad']

sql_columns = []
for col in columns_wind:
    if col not in geo_columns:
    # SQL create column statement for this column: 'column_name pgsql-type,'. Leave out constraints for now
    # to lowercase, underscore at uppercase
        name = change_case(col)
        sql_type = dtype_sqltype(df_wind[col].dtype, map_types)
        sql_columns.append(f"{name} {sql_type}")

sql_columns = ", \n ".join(sql_columns)

# leave out geo-columns -> added later on
# add primary key
sql_drop = f"DROP TABLE IF EXISTS {schema_name}.{table_name};"

sql_create = f"""
CREATE TABLE {schema_name}.{table_name} (
id bigint generated by default as identity primary key,
{sql_columns},
geom geometry(point, 4326) 
);""" 
    

In [ ]:
### Connect to the database
# downloaded certiticate
# Set connection details in .env

# Load environment variables from .env file
load_dotenv()

# Get connection parameters from environment variables
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
ssl_cert_path = os.getenv("SSL_CERT_PATH")

# Construct the connection string
conn_str = f"dbname={dbname} user={user} password={password} host={host} port={port} sslmode=require sslrootcert={ssl_cert_path}"

# Etablish connection object

#conn.close()
conn = psycopg2.connect(conn_str)

In [ ]:

print(conn)
print("Connection to the database successful!")
conn_cursor = conn.cursor()

# drop table if it already exists
conn_cursor.execute(sql_drop)

# table creation
conn_cursor.execute(sql_create)
conn.commit()

conn_cursor.close()

In [ ]:
### Selection works fine, as it does necessarily need conn.commit()
sql_test = "SELECT column_name FROM information_schema.columns WHERE table_schema = 'public' AND table_name = 'wind_extended';"

conn_cursor = conn.cursor()
conn_cursor.execute(sql_test)
results = conn_cursor.fetchall()
print(results)
conn_cursor.close()

In [168]:
### Helper Function for one preprocessed row without geodata 
### to generate column names and values

def row_data_to_sql(row_data, columns_data):
    ### Lists for column names as needed for the postgres-table and the values as given to the sql statement
    columns_sql = []
    values_sql = []

    for col in columns_data:
        val = row_data[col].values[0]
        # print(val)
        # print(type(val))
        
    # Test wether the column holds a value and is not empty
        if pd.notna(val):
        # add column name
            columns_sql.append(change_case(col))
        # Apply date to string transformation
            if isinstance(val, str):
                #print("is_str")
                values_sql.append(f"$${val}$$")   # add a pair of parentheses to keep for the join
            elif is_datetime64_any_dtype(val):
                #print("is_datetime")
                val = np.datetime_as_string(val, unit="D")
                values_sql.append(f"'{val}'")
            else:
                #print("is_float_bool")
                values_sql.append(str(val))     # cast to str without adding parentheses
    
    return columns_sql, values_sql

In [ ]:
### helper function to turn coordinates (right now EPSG:4326 only)
def row_geo_to_sql(row_geo, geo_columns, columns_sql, values_sql, epsg = 4326):   
    
    lat = geo_columns[0]
    lon = geo_columns[1]
    ### Append geom and its values to corresponding lists
    if pd.notna(row_geo[lon].values[0]) and pd.notna(row_geo[lat].values[0]):
        columns_sql.append('geom')
        wkt_point = f"POINT({row_geo[lon].values[0]} {row_geo[lat].values[0]})"
        val_geo = f"ST_GeomFromText('{wkt_point}',{epsg})"
        values_sql.append(val_geo)
    
    return columns_sql, values_sql

In [ ]:
### helper function to construct the INSERT query for one row 
#   from the columns_sql and values_sql lists

def join_insert_sql(columns_sql, values_sql):
    # join sql-column names and values respectively into a single string
    columns_sql = ", \n ".join(columns_sql) 
    values_sql = ", \n ".join(values_sql)

    # Create INSERT-Query for one row
    sql_insert = f"""INSERT INTO {schema_name}.{table_name} (
        {columns_sql} )
    VALUES (
        {values_sql}
        );
    """
    
    return sql_insert

In [166]:
### Function for all rows
def df_to_sql_insert(df_wind, conn_db):
    # join sql-column names and values respectively into a single string
    ### Loop through columns
    ### How should the geo-insert look like
    columns_wind = df_wind.columns

    ### column names where each name corresponds to one value (not true for db column geom) 
    columns_data = [col for col in columns_wind if col not in geo_columns]
    
    ### List to hold all INSERT Statements
    inserts_all = []
    
    for i in range(len(df_wind)):
        row_wind = df_wind.iloc[[i],:]
        
        row_data = row_wind[columns_data]    
        row_geo = row_wind[geo_columns]
        
        columns_sql, values_sql = row_data_to_sql(row_data, columns_data)
        columns_sql, values_sql = row_geo_to_sql(row_geo, geo_columns, columns_sql, values_sql)
        insert_sql = join_insert_sql(columns_sql, values_sql)
        
        inserts_all.append(insert_sql)
    
    inserts_all_sql = " \n ".join(inserts_all)
    
    # Establish a connection to the database    
    try:
        # Create a cursor
        cur = conn_db.cursor()
    
        # Execute your SQL statement
        cur.execute(inserts_all_sql)
    
        # Commit the transaction
        conn.commit()
    
    except Exception as e:
        # Handle the exception
        print(f"Error: {e}")
        conn.rollback()
    
    finally:
        # Close the cursor and connection
        cur.close()
        #conn.close()

In [170]:
df_one = df_wind.iloc[0:1000,:]
df_one

,NameWindpark,Hersteller,HerstellerId,Technologie,Typenbezeichnung,Nabenhoehe,Rotordurchmesser,Rotorblattenteisungssystem,AuflageAbschaltungLeistungsbegrenzung,AuflagenAbschaltungSchallimmissionsschutzNachts,...,FernsteuerbarkeitNb,FernsteuerbarkeitDv,FernsteuerbarkeitDr,Einspeisungsart,PraequalifiziertFuerRegelenergie,GenMastrNummer,Netzbetreiberzuordnungen,Hausnummer_nv,Weic_nv,Kraftwerksnummer_nv
0,Windpark Kreuzstein,ENERCON GmbH,None,Horizontalläufer,E-115,149.08,115.71,True,True,True,...,True,True,None,Volleinspeisung,None,SGE961876398816,None,False,False,False
1,Damsdorf,ENERCON GmbH,None,Horizontalläufer,E-115,135.40,115.70,None,True,False,...,True,True,None,Volleinspeisung,None,SGE918200258391,None,False,False,False
2,Windpark Kreuzstein,ENERCON GmbH,None,Horizontalläufer,E-115,149.08,115.71,True,True,False,...,True,True,None,Volleinspeisung,None,SGE961876398816,None,False,False,False
3,Windpark Kreuzstein,ENERCON GmbH,None,Horizontalläufer,E-115,149.08,115.71,True,True,True,...,True,True,None,Volleinspeisung,None,SGE961876398816,None,False,False,False
4,Windpark Heinsberg Pütt,Nordex SE,None,Horizontalläufer,N117,120.00,116.80,False,True,True,...,True,True,None,Volleinspeisung,None,SGE950157614196,None,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Metzenhausen,Vestas Deutschland GmbH,None,Horizontalläufer,V126,166.00,126.00,False,True,True,...,True,True,None,Volleinspeisung,None,SGE911375730136,None,False,True,False
996,None,NEG Micon Deutschland GmbH,None,Horizontalläufer,NM52/900,74.00,52.20,None,False,None,...,True,True,None,Volleinspeisung,None,None,None,True,True,False
997,Himmelberg,NEG Micon Deutschland GmbH,None,Horizontalläufer,M1500-600,46.00,43.00,None,False,None,...,True,True,None,Volleinspeisung,None,None,None,False,True,False
998,Windpark Lehmden,NEG Micon Deutschland GmbH,None,Horizontalläufer,NM52/900,74.00,52.20,None,False,None,...,True,True,None,Volleinspeisung,None,None,None,True,True,False


In [172]:
start_time = time.time()
df_to_sql_insert(df_one, conn)
end_time = time.time()

execution_time = end_time - start_time
print("Execution Time:", execution_time, "seconds")

Execution Time: 4.93808650970459 seconds


In [173]:
batch_size = 1000  # Set the desired batch size
total_rows = len(df_wind)

for i in range(1000, total_rows, batch_size):
    df_batch = df_wind[i:i+batch_size]
    df_to_sql_insert(df_batch, conn)

In [ ]:

#df_to_sql_insert(df_test,conn)

columns_wind = df_test.columns

### column names where each name corresponds to one value (not true for db column geom) 
columns_data = [col for col in columns_wind if col not in geo_columns]
    
    ### List to hold all INSERT Statements
inserts_all = []
    
for i in range(len(df_test)):
    row_wind = df_test.iloc[[i],:]
    
    row_data = row_wind[columns_data]    
    row_geo = row_wind[geo_columns]
        
    columns_sql, values_sql = row_data_to_sql(row_data, columns_data)
    
    columns_sql, values_sql = row_geo_to_sql(row_geo, geo_columns, columns_sql, values_sql)
    insert_sql = join_insert_sql(columns_sql, values_sql)
        
    inserts_all.append(insert_sql)

inserts_all_sql = " \n ".join(inserts_all)

# Establish a connection to the database    
try:
# Create a cursor
    cur = conn.cursor()
    
        # Execute your SQL statement
    cur.execute(inserts_all_sql)
    
        # Commit the transaction
    conn.commit()
    
except Exception as e:
        # Handle the exception
    print(f"Error: {e}")
    conn.rollback()
    
finally:
        # Close the cursor and connection
    cur.close()
    #conn.close()


In [ ]:
for i in range(len(df_wind)):
    df_wind.iloc[[i],:]